In [2]:
import math
from pathlib import Path
import sys
import numpy as np
import pandas as pd
from scipy import stats
import statistics

from sklearn.model_selection import train_test_split

In [3]:
import tcremb.data_proc as data_proc

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
outputs_path = 'data/data_preped/'

In [6]:
tcr_columns = {'single': ['cdr3aa','v','j','chain'],'paired': {'TRA': ['a_cdr3aa', 'TRAV', 'TRAJ'],'TRB': [ 'b_cdr3aa', 'TRBV', 'TRBJ']}}
tcr_columns = ['cdr3aa','v','j','chain']
clonotype_id_column = 'cloneId'
data_id= 'data_id'
annotation_id = 'annotId'
pairing_id = 'barcode'
annotation_tcr_id_columns_dict = {'TRA': 'cloneId','TRB': 'cloneId','TRA_TRB': {'TRA':'cloneId_TRA', 'TRB':'cloneId_TRB'}}

## Prototypes

### TRA

In [11]:
data_raw_path = 'data/olga_humanTRA_3000.txt'
data_output_path = outputs_path +'olga_humanTRA.txt'

In [12]:
data = pd.read_csv(data_raw_path,sep='\t',header=None)
data = data.rename({0:'cdr3nt',1:'cdr3aa',2:'v',3:'j'},axis=1)

In [13]:
data_preped = data_proc.remove_asterisk(data, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.remove_backslash(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_clones_data(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(3000, 4)
(3000, 4)


(2951, 4)

In [14]:
data_preped.to_csv(data_output_path, sep='\t',header=False)

### TRB

In [15]:
data_raw_path = 'data/olga_humanTRB_3000.txt'
data_output_path = outputs_path +'olga_humanTRB.txt'

In [16]:
data = pd.read_csv(data_raw_path,sep='\t',header=None)
data = data.rename({0:'cdr3nt',1:'cdr3aa',2:'v',3:'j'},axis=1)

In [17]:
data_preped = data_proc.remove_asterisk(data, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.remove_backslash(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_clones_data(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(3000, 4)
(3000, 4)


(3000, 4)

In [18]:
data_preped.to_csv(data_output_path, sep='\t',header=False)

## VDJdb

In [41]:
label = 'antigen.epitope'
label_s = 'antigen.species'

In [42]:
data_raw_path = 'data/vdjdb.slim.txt'
data_tra_output_path = outputs_path +'VDJdb_data_TRA.csv'
data_trb_output_path = outputs_path +'VDJdb_data_TRB.csv'
data_output_path = outputs_path +'VDJdb_data.csv'

In [43]:
data = pd.read_csv(data_raw_path,sep='\t')
data = data.rename({'cdr3':'cdr3aa','gene':'chain','v.segm':'v','j.segm':'j'},axis=1)




### Assign data id

In [44]:
data = data_proc.annot_id(data, data_id)

### data proc

In [45]:
data_preped = data_proc.remove_asterisk(data, tcr_columns)
data_preped = data_proc.remove_backslash(data_preped, tcr_columns)
data_preped = data_proc.filter_clones_data(data_preped, tcr_columns)
data_preped = data_preped[-data_preped['reference.id'].str.startswith('https://www.10xgenomics',na=False)].reset_index(drop=True)
data_preped = data_preped[data_preped['species']=='HomoSapiens']
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(75280, 17)
(74240, 17)


(36193, 17)

### frequent labels

In [19]:
data_preped = data_proc.freq_labels(label, data_id, data_preped)
data_preped = data_proc.freq_labels_tr_list(label, data_id, data_preped, [10,20,30,50,100,500,1000])
data_preped = data_proc.freq_labels(label_s, data_id, data_preped)
data_preped = data_proc.freq_labels_tr_list(label_s, data_id, data_preped, [10,20,30,50,100,500,1000])

### save

In [26]:
data_preped.to_csv(data_output_path, sep='\t',index=False)

### prep for atmtcr

In [39]:
atmtcr_trb_train_path = outputs_path + 'vdjdb_trb_atmtcr_train.csv'
atmtcr_trb_test_path = outputs_path + 'vdjdb_trb_atmtcr_test.csv'

In [27]:
data_preped_trb = data_preped[data_preped['chain']=='TRB'].reset_index(drop=True)

In [29]:
data_preped_trb_atmtcr = data_preped_trb.rename({label:'Epitope','cdr3aa':'TCR'},axis=1)[['Epitope','TCR']]

In [30]:
data_preped_trb_atmtcr['Binding Affinity']=1

In [37]:
data_preped_trb_atmtcr_train, data_preped_trb_atmtcr_test = train_test_split(data_preped_trb_atmtcr,test_size=0.3)

In [38]:
data_preped_trb_atmtcr_train

,Epitope,TCR,Binding Affinity
13263,LPRRSGAAGA,CASSDESWGEDEQFF,1
10540,GILGFVFTL,CASSMGIYGYTF,1
22616,GLCTLVAML,CASSSDSYEQYF,1
7140,YLEPGPVTV,CASSIGAPYEQYF,1
21865,ELAGIGILTV,CASSIAAVSLANYGYTF,1
...,...,...,...
23657,QYIKWPWYI,CASSIVLSEKLFF,1
16681,NLVPMVATV,CASSSIRTGTFDYEQYF,1
503,DATYQRTRALVR,CATSSEATGVGETQYF,1
17437,KLDDKDPNF,CASSVGVSNQPQHF,1


In [40]:
data_preped_trb_atmtcr_train.to_csv(atmtcr_trb_train_path, index=False)
data_preped_trb_atmtcr_test.to_csv(atmtcr_trb_test_path, index=False)

## 10x donor 1

In [7]:
donor = 'donor1'
label = 'top_tetramer'

In [11]:
barcodes_path = '/home/ykremlyakova/projects/Archive/tcr_emb_mirpy_old/data/10x_' + donor + '/filtered_feature_bc_matrix/barcodes.tsv'
features_path = '/home/ykremlyakova/projects/Archive/tcr_emb_mirpy_old/data/10x_' + donor + '/filtered_feature_bc_matrix/features.tsv'
matrix_path = '/home/ykremlyakova/projects/Archive/tcr_emb_mirpy_old/data/10x_' + donor + '/filtered_feature_bc_matrix/matrix.mtx'
annot_path = '/home/ykremlyakova/projects/Archive/tcr_emb_mirpy_old/data/10x_' + donor + '/vdj_v1_hs_aggregated_' + donor + '_all_contig_annotations.csv'

In [12]:
matrix_path_preped = outputs_path + '10x_matrix_' + donor + '.txt'
data_output_path = outputs_path + '10x_annot_data_' + donor + '.txt'
data_paired_output_path = outputs_path + '10x_annot_data_paired' + donor + '.txt'

In [13]:
matrix_path_preped

'data/data_preped/10x_matrix_donor1.txt'

### 10x preprocessing - get matrix with values

### Get top tetramer

In [16]:
matrix = pd.read_csv(matrix_path_preped,sep='\t')
barcode_tetramer = data_proc.get_barcode_top_tetramer(matrix)
barcode_tetramer = barcode_tetramer[barcode_tetramer['top_tetramer']!='KLGGALQAK']
barcode_tetramer = barcode_tetramer[barcode_tetramer['count']>4]

### prep annot

In [17]:
data_annot = pd.read_csv(annot_path,sep=',')

In [22]:
data = pd.merge(data_annot, barcode_tetramer, on = pairing_id)

In [23]:
data = data.rename({'cdr3':'cdr3aa','v_gene':'v','j_gene':'j'},axis=1)

In [24]:
data = data_proc.annot_id(data, data_id)

In [25]:
data_preped = data_proc.remove_asterisk(data, tcr_columns)
data_preped = data_proc.remove_backslash(data_preped, tcr_columns)
data_preped = data_proc.filter_clones_data(data_preped, tcr_columns)
data_preped = data_preped[data_preped['high_confidence']==True]
data_preped = data_preped[data_preped['is_cell']==True]
data_preped = data_preped[data_preped['chain']!='Multi']

data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(56823, 21)
(36654, 21)


(27651, 21)

### Only 1 barcoce

In [26]:
data_preped = data_preped.sort_values(['umis','reads'],ascending=False).drop_duplicates(['barcode','chain'])
print(data_preped.shape)

(25673, 21)


### frequent labels

In [32]:
label_high = str(label + '_freq')

In [28]:
data_preped = data_proc.freq_labels(label, data_id, data_preped, n = 8)
data_preped = data_proc.freq_labels_tr_list(label, data_id, data_preped, [10,20,30,50,100,500,1000])

### save 

In [29]:
data_preped.to_csv(data_output_path,sep='\t', index = False)

In [31]:
data_preped

,barcode,is_cell,contig_id,high_confidence,length,chain,v,d_gene,j,c_gene,...,count,data_id,top_tetramer_freq,top_tetramer_freq_10,top_tetramer_freq_20,top_tetramer_freq_30,top_tetramer_freq_50,top_tetramer_freq_100,top_tetramer_freq_500,top_tetramer_freq_1000
13350,CTCTAATAGACTGTAA-23,True,CTCTAATAGACTGTAA-23_contig_1,True,701,TRB,TRBV7-6,TRBD2,TRBJ2-3,TRBC2,...,12,25812,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
12693,CTAGTGACACGTAAGG-1,True,CTAGTGACACGTAAGG-1_contig_1,True,734,TRB,TRBV7-2,TRBD1,TRBJ2-7,TRBC2,...,5,24499,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
23406,TCAGGATCAATGAAAC-2,True,TCAGGATCAATGAAAC-2_contig_1,True,672,TRB,TRBV4-1,TRBD1,TRBJ2-3,TRBC2,...,8,44988,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
20100,GTATTCTTCCTTTCTC-8,True,GTATTCTTCCTTTCTC-8_contig_1,True,699,TRB,TRBV27,NaN,TRBJ2-7,TRBC2,...,5,38742,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK
24199,TCTATTGCACAGACTT-11,True,TCTATTGCACAGACTT-11_contig_1,True,903,TRB,TRBV4-1,TRBD1,TRBJ2-3,TRBC2,...,7,46508,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2735,ACTGAACCAACGATGG-26,True,ACTGAACCAACGATGG-26_contig_2,True,484,TRA,TRAV35,NaN,TRAJ49,TRAC,...,141,5307,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK
11065,CGCTTCATCCTTGACC-24,True,CGCTTCATCCTTGACC-24_contig_2,True,501,TRA,TRAV35,NaN,TRAJ49,TRAC,...,7,21378,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
21890,TACCTATCACGACGAA-40,True,TACCTATCACGACGAA-40_contig_2,True,605,TRA,TRAV14DV4,NaN,TRAJ45,TRAC,...,15,42114,other,FLRGRAYGL,FLRGRAYGL,FLRGRAYGL,FLRGRAYGL,other,other,other
23774,TCGAGGCGTCTCTTTA-30,True,TCGAGGCGTCTCTTTA-30_contig_4,True,505,TRA,TRAV35,NaN,TRAJ49,TRAC,...,242,45709,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK


### Map pairs

In [33]:
data_a = data_preped[data_preped['chain']=='TRA'][['cdr3aa','v','j']+[ pairing_id, label, label_high]]
data_a = data_a.rename({'cdr3aa':'a_cdr3aa','v':'TRAV','j':'TRAJ'},axis=1)
data_b = data_preped[data_preped['chain']=='TRB'][['cdr3aa','v','j'] + [pairing_id, label, label_high]]
data_b = data_b.rename({'cdr3aa':'b_cdr3aa','v':'TRBV','j':'TRBJ'},axis=1)


In [34]:
data_paired = data_a.merge(data_b)
data_paired

,a_cdr3aa,TRAV,TRAJ,barcode,top_tetramer,top_tetramer_freq,b_cdr3aa,TRBV,TRBJ
0,CAVTDSWGKLQF,TRAV21,TRAJ24,GCAAACTAGAAGCCCA-26,AVFDRKSDAK,AVFDRKSDAK,CASKPTSTDTQYF,TRBV27,TRBJ2-3
1,CAYRSAKDSSYKLIF,TRAV38-2DV8,TRAJ12,AGAGCTTCAAGCCGTC-8,AVFDRKSDAK,AVFDRKSDAK,CASSSHPLATFTGELFF,TRBV7-2,TRBJ2-2
2,CAVDPMDTGRRALTF,TRAV8-3,TRAJ5,GTCATTTTCACCTCGT-27,AVFDRKSDAK,AVFDRKSDAK,CASSLGFTDTQYF,TRBV27,TRBJ2-3
3,CAVHGYGQNFVF,TRAV21,TRAJ26,CGACTTCCAAACCCAT-23,AVFDRKSDAK,AVFDRKSDAK,CASSQEWLAVSTDTQYF,TRBV4-3,TRBJ2-3
4,CATGDNYGQNFVF,TRAV17,TRAJ26,ACGGGCTAGGAGTACC-28,AVFDRKSDAK,AVFDRKSDAK,CASSDGQGLEQYF,TRBV27,TRBJ2-7
...,...,...,...,...,...,...,...,...,...
11695,CAGHTGNQFYF,TRAV35,TRAJ49,CGCTATCCAGTCGATT-4,IVTDFSVIK,IVTDFSVIK,CASSWGGGSHYGYTF,TRBV11-2,TRBJ1-2
11696,CAGHTGNQFYF,TRAV35,TRAJ49,ACTGAACCAACGATGG-26,IVTDFSVIK,IVTDFSVIK,CASSWGGGSHYGYTF,TRBV11-2,TRBJ1-2
11697,CAGHTGNQFYF,TRAV35,TRAJ49,CGCTTCATCCTTGACC-24,AVFDRKSDAK,AVFDRKSDAK,CSAQWTPGGNQPQHF,TRBV20-1,TRBJ1-5
11698,CAMREGVSGGGADGLTF,TRAV14DV4,TRAJ45,TACCTATCACGACGAA-40,FLRGRAYGL,other,CASSPPEREITDTQYF,TRBV18,TRBJ2-3


### save pairs

In [35]:
data_paired.to_csv(data_paired_output_path,sep='\t', index = False)